# Registration

# Anatomical >> Standard space registration

In [ ]:
import glob
import os


data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'

reference_path = '/mnt/d/Data/Chick/Imprinting/analysis/template/'

sub_dirs = glob.glob('%s/sub*'%(data_path))

for curr_sub in sub_dirs:
    
    reg_directory = os.path.join(curr_sub,'reg')
    
    print(reg_directory)
    os.makedirs(reg_directory)
    anat_dir = os.path.join(curr_sub,'anat')
    os.system('fslmaths %s/anat %s/highres'%(anat_dir,reg_directory))
    os.system('fslmaths %s/anat_brain %s/highres_head'%(anat_dir,reg_directory))
    os.system('fslmaths %s/standard %s/standard'%(reference_path,reg_directory))
    os.system('fslmaths %s/standard %s/standard_head'%(reference_path,reg_directory))
    os.system('fslmaths %s/standard_mask %s/standard_mask'%(reference_path,reg_directory))
    
    # change directory to the reg 
    os.chdir(reg_directory)
    
    #highres 2 standard    
    os.system('flirt -in highres_head -ref standard -out highres2standard -omat highres2standard.mat -bins 256 -cost corratio -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -dof 6  -interp trilinear')
    #os.system('flirt -in highres -ref standard -out highres2standard -omat highres2standard.mat -cost corratio -dof 6 -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -interp trilinear')

    os.system('fnirt --iout=highres2standard_head --in=highres_head --aff=highres2standard.mat --cout=highres2standard_warp --iout=highres2standard --jout=highres2highres_jac --ref=standard_head --refmask=standard_mask --warpres=6,6,6 --regmod=membrane_energy')
    os.system('applywarp -i highres -r standard -o highres2standard -w highres2standard_warp')
    os.system('convert_xfm -inverse -omat standard2highres.mat highres2standard.mat')

              
    os.system('slicer highres2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard1.png ; slicer standard highres2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png highres2standard2.png ; pngappend highres2standard1.png - highres2standard2.png highres2standard.png; rm -f sl?.png highres2standard2.png')
    os.system('rm highres2standard1.png')

# Plot Anatomical >> Atlas registration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'
reg_dirs = glob.glob('%s/sub*/reg'%(data_path))

for curr_reg in reg_dirs[0]:
    
    #print(curr_reg)
    print(curr_reg[-9:-4])
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'highres2standard.png' )))

# copy reg to single days folder and create example_func file and transfer bold to the results directory

In [ ]:
# first the code will copy the reg directory for ach day folder. Then, it will remove dummy scan from data and 
# save the bold file in results direcoty. Later the middle volume willbe extracted as example_func file and save in reg directory 
import glob
import os
import subprocess

dummy_scan = 5#number of volumes for delete

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis/'

sub_dirs = glob.glob('%s/sub*/2*'%(data_path))

for curr_sub in sub_dirs:
    print(curr_sub)
    
    # copy reg directoy to individual days
    reg_str = os.path.join(os.path.dirname(curr_sub),'reg')
    reg_dst = os.path.join(curr_sub,'results/reg')
    os.makedirs(reg_dst)
    os.system('cp -R %s/* %s'%(reg_str,reg_dst))
    
    # read number of volumes for each run and remove first five volume
    cur_bold = os.path.join(curr_sub,'rest/bold.nii.gz')
    results_folder = os.path.join(curr_sub,'results')
    nVloume = subprocess.check_output("fslnvols %s"%(cur_bold), shell=True).decode('utf-8')
    os.system("fslroi %s %s/bold.nii.gz %s %s"%(cur_bold,results_folder,dummy_scan,int(nVloume)-int(dummy_scan)))
    
    # create example_func file: the midle volume of session
    nVloume = subprocess.check_output("fslnvols %s/bold.nii.gz"%(results_folder), shell=True).decode('utf-8')
    os.system("fslroi %s/bold %s/example_func %s %s"%(results_folder,reg_dst,round((int(nVloume))/2),1))
    


# Functional >> Anatomical space registration
<br>



In [ ]:
import glob
import os
import subprocess
data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'
sub_dirs = glob.glob('%s/sub*'%(data_path))

for curr_sub in sub_dirs:
    
    reg_dir = os.path.join(curr_sub,'reg')
    print(reg_dir)
    
    cur_bold = os.path.join(curr_sub,'results/bold.nii.gz')
    nVloume = subprocess.check_output("fslnvols %s"%(cur_bold), shell=True).decode('utf-8')
    os.system("fslroi %s %s/example_func %s %s"%(cur_bold,reg_dir,round((int(nVloume))/2),1))
    # change path
    os.chdir(reg_dir)

    #FSL
    # register func2highres
    os.system('flirt -in example_func -ref highres -out example_func2highres -omat example_func2highres.mat -bins 256 -cost mutualinfo -searchrx 0 0 -searchry 0 0 -searchrz 0 0 -dof 6  -interp trilinear')
    os.system('convert_xfm -inverse -omat highres2example_func.mat example_func2highres.mat')
    os.system('slicer example_func2highres highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres1.png ; slicer highres example_func2highres -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2highres2.png ; pngappend example_func2highres1.png - example_func2highres2.png example_func2highres.png; rm -f sl?.png example_func2highres2.png')
    os.system('rm example_func2highres1.png')


# Plot example_func >> anatomical space registeration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Chick/Imprinting/analysis'

reg_dirs = glob.glob('%s/sub*/reg'%(data_path))
for curr_reg in reg_dirs:
    
    print(curr_reg)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    print(curr_reg.split('/')[-2])
    #imgplot = plt.imshow(img)
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'example_func2highres.png' )))


# conver highres mask to functional space

In [ ]:
import glob
import os

data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'
sub_dirs = glob.glob('%s/sub*/reg'%(data_path))

for curr_sub in sub_dirs:

    print(curr_sub)
    
    # change path
    os.chdir(curr_sub)
    os.system('fslmaths highres_head.nii.gz -div highres_head.nii.gz highres_mask')
    os.system('flirt -in highres_mask.nii.gz -ref example_func -applyxfm -init highres2example_func.mat -out example_func_mask')
    os.system('fslmaths example_func_mask -thr 0.25 example_func_mask')
    os.system('fslmaths example_func_mask -div example_func_mask example_func_mask')

# plot mask in functional spaces

In [ ]:
import glob
import os
import pandas as pd
from nilearn.image import new_img_like, load_img, get_data
import numpy as np
from nilearn import plotting

data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'
reg_dirs = glob.glob('%s/sub*/reg'%(data_path))

for curr_reg in reg_dirs:
    
    #print(reg_dir)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    print(curr_reg.split('/')[-2])
    mask_img = load_img(os.path.join(curr_reg, 'example_func_mask.nii.gz'))
    # .astype() makes a copy.
    process_mask = get_data(mask_img).astype(np.int)
    #picked_slice = 29
    #process_mask[:,:,:picked_slice] = 0
    #process_mask[:,:,(picked_slice+1):] = 0
    #process_mask[:, 30:] = 0
    process_mask_img = new_img_like(mask_img, process_mask)

    plotting.plot_roi(process_mask_img, bg_img=os.path.join(curr_reg, 'example_func.nii.gz'),
                      cmap='Paired')
    plotting.show()


# concatenate functional >> highres and highres >> standrad

In [ ]:
import glob
import os
import subprocess
data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'
sub_dirs = glob.glob('%s/sub*'%(data_path))

for curr_sub in sub_dirs:
    
    reg_dir = os.path.join(curr_sub,'reg')
    print(reg_dir)
    # change path
    os.chdir(reg_dir)
    
    # convert tranform matrix 
    os.system('convert_xfm -omat example_func2standard.mat -concat highres2standard.mat example_func2highres.mat')

    os.system('convertwarp --ref=standard --premat=example_func2highres.mat --warp1=highres2standard_warp --out=example_func2standard_warp')

    os.system('applywarp --ref=standard --in=example_func --out=example_func2standard --warp=example_func2standard_warp')

    os.system('convert_xfm -inverse -omat standard2example_func.mat example_func2standard.mat')

    os.system('slicer example_func2standard standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard1.png ; /usr/local/fsl/bin/slicer standard example_func2standard -s 2 -x 0.35 sla.png -x 0.45 slb.png -x 0.55 slc.png -x 0.65 sld.png -y 0.35 sle.png -y 0.45 slf.png -y 0.55 slg.png -y 0.65 slh.png -z 0.35 sli.png -z 0.45 slj.png -z 0.55 slk.png -z 0.65 sll.png ; /usr/local/fsl/bin/pngappend sla.png + slb.png + slc.png + sld.png + sle.png + slf.png + slg.png + slh.png + sli.png + slj.png + slk.png + sll.png example_func2standard2.png ; /usr/local/fsl/bin/pngappend example_func2standard1.png - example_func2standard2.png example_func2standard.png; /bin/rm -f sl?.png example_func2standard2.png')
    

# Plot example_func >> standard space registeration results

In [ ]:
%matplotlib inline
%pylab inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Chick/Imprinting/analysis/'

reg_dirs = glob.glob('%s/sub*/reg'%(data_path))
for curr_reg in reg_dirs:
    
    #print(reg_dir)
    #img = mpimg.imread(os.path.join(reg_dir, 'example_func2standard.png' ))
    print(curr_reg.split('/')[-2])
    #imgplot = plt.imshow(img)
    plt.show()
    display(Image(filename=os.path.join(curr_reg, 'example_func2standard.png' )))
